In [87]:
%matplotlib inline
import os
import sys
import pandas as pd
import numpy as np
import sqlite3
import urllib

To download the data file to your local directory, run the command below. The file is ~1Gb, therefore it might take some time. 
!!!! You just have to run this command once!!!

In [5]:
%%bash
wget -O hampt_rd_data.sqlite https://osf.io/mr7jx/?action=download 

--2018-07-27 10:20:58--  https://osf.io/mr7jx/?action=download
Resolving osf.io (osf.io)... 35.190.84.173
Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.osf.io/v1/resources/9yvs4/providers/osfstorage/5b59db88e42024000d38d962?action=download&version=1&direct [following]
--2018-07-27 10:20:58--  https://files.osf.io/v1/resources/9yvs4/providers/osfstorage/5b59db88e42024000d38d962?action=download&version=1&direct
Resolving files.osf.io (files.osf.io)... 35.186.214.196
Connecting to files.osf.io (files.osf.io)|35.186.214.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1096573952 (1.0G) [application/octet-stream]
Saving to: ‘hampt_rd_data.sqlite’

     0K .......... .......... .......... .......... ..........  0% 1.21M 14m27s
    50K .......... .......... .......... .......... ..........  0% 2.30M 11m1s
   100K .......... .......... .......... .......... ..........  0% 3.6

In [ ]:
db_filepath = ("hampt_rd_data.sqlite")

In [71]:
def get_db_table_as_df(name, sql="""SELECT * FROM {};""", date_col=None, dbfilename=db_filepath):
    con = sqlite3.connect(dbfilename)
    sql = sql.format(name)
    if name == 'datavalues':
        date_col = 'Datetime'
    df = pd.read_sql(sql, con, parse_dates=date_col)
    if name == 'datavalues':
        df = make_date_index(df, 'Datetime')
    return df

In [72]:
def get_id(typ, data):
    """
    gets either the siteid or variableid from the db
    :param typ: String. Either "Site" or "Variable"
    :param data: Dict. the site or variable data
    :return: int. id of site or variable
    """
    data_df = pd.DataFrame(data, index=[0])
    code_name = '{}Code'.format(typ)
    table_name = '{}s'.format(typ.lower())
    id_name = '{}ID'.format(typ)
    code = data[code_name]
    check_by = [code_name]
    table = get_db_table_as_df(table_name)
    id_row = table[table[code_name] == code]
    id_num = id_row[id_name].values[0]
    return id_num

In [73]:
def make_date_index(df, field, fmt=None):
    df.loc[:, field] = pd.to_datetime(df.loc[:, field], format=fmt)
    df.set_index(field, drop=True, inplace=True)
    return df

In [74]:
def get_table_for_variable_code(variable_code, site_id=None, start_date=None, end_date=None):
    var_id = get_id('Variable', {'VariableCode': variable_code})
    table_name = 'datavalues'
    sql = """SELECT * FROM {} WHERE VariableID={};""".format(table_name, var_id)
    if start_date or end_date:
        if not start_date:
            start_date = '1900-01-01'
        elif not end_date:
            end_date = '2100-01-01'
        sql = """SELECT * FROM {} WHERE VariableID={} AND Datetime BETWEEN '{}' AND '{}';""".format(
            table_name,
            var_id,
            start_date,
            end_date
        )

    df = get_db_table_as_df(table_name, sql=sql)
    df = df.sort_index()
    if site_id:
        df = df[df['SiteID'] == site_id]
    return df

In [75]:
def round_down_near_24(datetimes): # round down the times near midnight so the tide levels stay on the correct day
    close_time_idx = datetimes.indexer_between_time('23:29', '23:59')
    adjusted_times = datetimes[close_time_idx] - pd.Timedelta(minutes=15)
    dt = pd.Series(datetimes)
    dt[close_time_idx] = adjusted_times
    dt = pd.DatetimeIndex(dt)
    return dt

In [76]:
def cln_n_rnd_times(df):
    for i in range(df.shape[1]):
        datetimes = df.iloc[:, i]
        times = pd.DatetimeIndex(datetimes)
        rnd_dn = round_down_near_24(times)
        df.iloc[:, i] = rnd_dn
    return df

In [77]:
def pivot_dv_df(df):
    return df.pivot(columns='SiteID', values='Value')

In [78]:
def rename_cols(df, var_abbrev):
    if var_abbrev != "":
        new_df = df.copy()
        cols = df.columns.tolist()
        new_cols = ['{}-{}'.format(var_abbrev, c) for c in cols]
        new_df.columns = new_cols
        return new_df
    else:
        return df

In [79]:
def filter_max_rain_time_dfs(rain_daily_df, time_df):
    timemx_filt = pd.DataFrame(np.where(rain_daily_df>0, time_df, np.datetime64('NaT')))
    timemx_filt.columns = time_df.columns
    timemx_filt.index = time_df.index
    return timemx_filt

In [80]:
def tide_when_rain_max(rn_mx_time_df):
    td_df = get_table_for_variable_code('six_min_tide')
    try:
        td_df = pivot_dv_df(td_df)
    except:
        td_df = remove_duplicates(td_df)
        td_df = pivot_dv_df(td_df)
    td_df = td_df.resample('15T').mean()
    rn_mx_time_rnd = cln_n_rnd_times(rn_mx_time_df)
    l = []
    for c in rn_mx_time_rnd.columns:
        times = rn_mx_time_rnd.loc[:, c]
        tides = td_df.loc[times].resample('D').max()
        rain_var = c.split('_')[0]
        rain_site = c.split('-')[-1]
        new_cols = ['{}-{}_td-{}'.format(rain_var, rain_site, col) for col in tides.columns]
        tides.columns = new_cols
        l.append(tides)
    new_df = pd.concat(l, axis=1)
    new_df.sort_index(inplace=True)
    return new_df

In [81]:
def remove_duplicates(df):
    siteids = df['SiteID'].unique()
    df.reset_index(inplace=True)
    print df.shape
    non_duplicated = list()
    for site in siteids:
        df_site = df[df['SiteID'] == site]
        df_site_vals = df_site['Datetime']
        df_no_dups = ~df_site_vals.duplicated()
        df_no_dups_idx = df_site[df_no_dups].index
        non_duplicated.extend(df_no_dups_idx.tolist())
    df = df.loc[non_duplicated]
    df.set_index('Datetime', drop=True, inplace=True)
    print df.shape
    return df

In [82]:
def daily_pivot_table(var_code, agg_function, abbreviation):    
    df = get_table_for_variable_code(var_code)
    try:
        dfp = pivot_dv_df(df)
    except ValueError:
        df = remove_duplicates(df)
        dfp = pivot_dv_df(df)
    dfd = dfp.resample('D')
    aggrd = dfd.agg(agg_function)
    rnmed = rename_cols(aggrd, abbreviation)
    return rnmed

#  Rainfall

In [83]:
# get rainfall data at 15 min interval
rain_df = get_table_for_variable_code('rainfall')

## Daily Rainfall

In [84]:
rain_daily15 = daily_pivot_table('rainfall', np.sum, '')
rain_daily = daily_pivot_table('daily_rainfall', np.sum, '')
rain_daily_comb_no_name = pd.concat([rain_daily, rain_daily15], axis=1)
rain_daily_comb_named = rename_cols(rain_daily_comb_no_name, 'rd')
rain_daily_comb_named.head()

,rd-19,rd-20,rd-1,rd-2,rd-7,rd-11,rd-12,rd-13,rd-14,rd-15,rd-16,rd-21
Datetime,,,,,,,,,,,,
2010-01-01,0.01,0.01,0.0,0.0,0.05,0.03,0.06,0.02,0.01,0.01,0.02,0.03
2010-01-02,0.00,0.00,0.0,0.0,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00
2010-01-03,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00
2010-01-04,0.00,0.00,0.0,0.0,0.10,0.10,0.00,0.11,0.00,0.00,0.00,0.00
2010-01-05,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.12,0.00,0.00,0.11


In [85]:
rain_daily_comb_named.tail()

,rd-19,rd-20,rd-1,rd-2,rd-7,rd-11,rd-12,rd-13,rd-14,rd-15,rd-16,rd-21
Datetime,,,,,,,,,,,,
2016-12-28,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00
2016-12-29,0.19,0.20,0.18,0.0,0.0,0.2,0.0,0.19,0.16,0.26,0.3,0.24
2016-12-30,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00
2016-12-31,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00
2017-01-01,0.02,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Hourly Rainfall

In [86]:
rain15 = pivot_dv_df(rain_df)
rain_hourly_totals = rain15.rolling(window='H').sum()
rhr_mx = rain_hourly_totals.resample('D').max()
rhr_mx = rename_cols(rhr_mx, 'rhrmx')
rhr_mx.head()

,rhrmx-1,rhrmx-2,rhrmx-7,rhrmx-11,rhrmx-12,rhrmx-13,rhrmx-14,rhrmx-15,rhrmx-16,rhrmx-21
Datetime,,,,,,,,,,
2010-01-01,NaN,NaN,0.04,0.01,0.04,0.01,0.01,0.01,0.01,0.02
2010-01-02,NaN,NaN,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00
2010-01-03,NaN,NaN,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00
2010-01-04,NaN,NaN,0.10,0.10,0.00,0.11,0.00,0.00,0.00,0.00
2010-01-05,NaN,NaN,0.00,0.00,0.00,0.00,0.12,0.00,0.00,0.11


In [ ]:
rhr_timemx = rain_hourly_totals.groupby(pd.TimeGrouper('D')).idxmax()
rhr_timemx = rename_cols(rhr_timemx, 'rhr_mxtime')
rhr_timemx = filter_max_rain_time_dfs(rain_daily15, rhr_timemx)
rhr_timemx.head()

## 15-min max rainfall

In [ ]:
r15_mx = rain15.resample('D').max()
r15_mx = rename_cols(r15_mx, 'r15mx')
r15_mx.head()

In [ ]:
r15_timemx = rain15.groupby(pd.TimeGrouper('D')).idxmax()
r15_timemx = rename_cols(r15_timemx, 'r15_mxtime')
r15_timemx = filter_max_rain_time_dfs(rain_daily15, r15_timemx)
r15_timemx.head()

### Rain prev 3 days

In [ ]:
rain_prev_3_days = rain_daily_comb_no_name.shift(1).rolling(window=3).sum()
rain_prev_3_days = rename_cols(rain_prev_3_days, 'r3d')
rain_prev_3_days.head()

In [ ]:
rain_daily_comb_named['rd-14'][rain_daily_comb_named['rd-14']<0]

In [ ]:
rain15.loc['2014-06-24']

In [ ]:
rain_prev_3_days.plot.box()

#  Groundwater

In [ ]:
gw_df = daily_pivot_table('shallow_well_depth', np.mean, 'gw_av')
gw_df.head()

#  Tide

## Average daily tide

In [ ]:
tide_df = daily_pivot_table('six_min_tide', np.mean, 'td_av')
tide_df.head()

##  Tide when rain is at max

In [ ]:
td_r15mx = tide_when_rain_max(r15_timemx)
td_r15mx.head()

In [ ]:
td_rhrmx = tide_when_rain_max(rhr_timemx)
td_rhrmx.head()

## HI/LOs

In [ ]:
hilos = []
for v in ['high_tide', 'high_high_tide', 'low_tide', 'low_low_tide']:
    hilos.append(daily_pivot_table(v, np.mean, "".join(w[0] for w in v.split('_'))))

In [ ]:
hilo_df = pd.concat(hilos, axis=1)
hilo_df.head()

#  Wind

In [ ]:
wind_dfs = []
for v in ['WDF2', 'WSF2', 'AWDR', 'AWND', 'WGF6', 'WSF6', 'WDF6', 'WS2min', 'WD2min']:
    if v == 'WSF6':
        abbr = 'AWND'
    elif v == 'WDF6':
        abbr = 'AWDR'
    elif v == 'WS2min':
        abbr = 'AWND'
    elif v == 'WD2min':
        abbr = 'AWDR'
    else:
        abbr = v
    wind_dfs.append(daily_pivot_table(v, np.mean, abbr))
all_wind = pd.concat(wind_dfs, axis=1)
all_wind.head()

In [ ]:
feature_df = pd.concat([all_wind, hilo_df, td_r15mx, td_rhrmx, tide_df, gw_df, r15_mx, rhr_mx, rain_daily_comb_named, rain_prev_3_days], axis=1)
feature_df = feature_df.loc['2010-09-15':'2016-10-15']
feature_df.head()


### Save Daily Observations to DB

In [ ]:
# con = sqlite3.connect(db_filename)
# feature_df.to_sql(con=con, name="nor_daily_observations", if_exists="replace")
feature_df.to_csv('nor_daily_observations_standalone.csv')